<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/project_configuration/webhooks.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/master/examples/project_configuration/webhooks.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Webhook Configuration

Webhooks are supported for the following events:
* label_created
* label_updated
* label_deleted
* review_created
* review_updated

In [ ]:
!pip install labelbox
!pip install requests
!pip install hmac
!pip install hashlib
!pip install flask
!pip install Werkzeug

In [ ]:
from labelbox import Client, Webhook
from flask import Flask, request
import hmac
import hashlib
import threading
from werkzeug.serving import run_simple
import json
import requests
import os
from getpass import getpass
import socket

In [ ]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler

    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [ ]:
# Set this to a project that you want to use for the webhook
PROJECT_ID = ""
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [ ]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

In [ ]:
# We are using port 3001 for this example.
# Feel free to set to whatever port you want
WH_PORT = 3001

### Configure NGROK (Optional)
* If you do not have a public ip address then follow along

1. Create an account:
    https://dashboard.ngrok.com/get-started/setup
2. Download ngrok and extract the zip file
3. Add ngrok to your path
4. Add the authtoken `ngrok authtoken <token>`

In [ ]:
if not COLAB:
    os.system(f"ngrok http {WH_PORT} &")

### Configure server to receive requests

In [ ]:
# This can be any secret that matches your webhook config (we will set later)
secret = b"example_secret"

In [ ]:
app = Flask(__name__)


@app.route("/")
def hello_world():
    return "Hello, World!"


@app.route("/webhook-endpoint", methods=["POST"])
def print_webhook_info():
    payload = request.data
    computed_signature = hmac.new(secret, msg=payload,
                                  digestmod=hashlib.sha1).hexdigest()
    if request.headers["X-Hub-Signature"] != "sha1=" + computed_signature:
        print(
            "Error: computed_signature does not match signature provided in the headers"
        )
        return "Error", 500, 200

    print("=========== New Webhook Delivery ============")
    print("Delivery ID: %s" % request.headers["X-Labelbox-Id"])
    print("Event: %s" % request.headers["X-Labelbox-Event"])
    print("Payload: %s" %
          json.dumps(json.loads(payload.decode("utf8")), indent=4))
    return "Success"


thread = threading.Thread(target=lambda: run_simple("0.0.0.0", WH_PORT, app))
thread.start()

#### Test server

In [ ]:
print(requests.get("http://localhost:3001").text)

### Create Webhook

- Set ip address if your ip is publicly accessible.
- Otherwise use the following to get ngrok public_url

In [ ]:
if not COLAB:
    res = requests.get("http://localhost:4040/api/tunnels")
    assert (res.status_code == 200
           ), f"ngrok probably isn't running. {res.status_code}, {res.text}"
    tunnels = res.json()["tunnels"]
    tunnel = [
        t for t in tunnels if t["config"]["addr"].split(":")[-1] == str(WH_PORT)
    ]
    tunnel = tunnel[0]  # Should only be one..
    public_url = tunnel["public_url"]
else:
    public_url = f"http://{socket.gethostbyname(socket.getfqdn(socket.gethostname()))}"
print(public_url)

In [ ]:
# Set project to limit the scope to a single project
project = client.get_project(PROJECT_ID)
topics = {topic.value for topic in Webhook.Topic}
# For Global Webhooks (Global = per workspace) project = None
webhook = Webhook.create(client,
                         topics=topics,
                         url=public_url,
                         secret=secret.decode(),
                         project=project)

In [ ]:
# Ok so we should be configured assuming everything is setup correctly.
# Go to the following url and make a new label to see if it works
print(f"https://app.labelbox.com/projects/{PROJECT_ID}")

### Update Webhook

In [ ]:
# url, topics, and status can all be updated
updated_url = f"{public_url}/webhook-endpoint"
print(updated_url)
webhook.update(url=updated_url)
# Go to the following url and try one last time.
# Any supported action should work (create, delete, update a label, or create, update, or delete a review)
print(f"https://app.labelbox.com/projects/{PROJECT_ID}")

### List and delete all webhooks

In [ ]:
# DELETE:
webhook.update(status=Webhook.Status.INACTIVE.value)

# FETCH ALL WEBHOOKS:
org = client.get_organization()
webhooks = org.webhooks()

# Run this to clear all.
# WARNING!!! THIS WILL DELETE ALL WEBHOOKS FOR YOUR ORG
# ONLY RUN THIS IS YOU KNOW WHAT YOU ARE DOING.
# for webhook in webhooks:
#    print(webhook)
#    webhook.update(status = Webhook.Status.INACTIVE.value)